### Service API ### 

#### Install Lib ####
- 1.requests
 - pip install requests
- 2.flask
 - pip install flask
- 3.flask-restful
 - pip install flask-restful
 
#### polyglot ####
 - !polyglot download sentiment2.en

In [ ]:
### -- Import Lib -- ###

#!/usr/bin/env python
# -*- coding: utf-8 -*- 

from flask import Flask, jsonify, request
from flask_restful import Resource, Api 
from json import dumps
from polyglot.detect import Detector
from polyglot.text import Text

In [ ]:
### -- Select Lexicon -- ###

# lexicon dir : /sentiment_file/lexicon/lexicon.txt

lexicon = 'officeM'


In [ ]:
### -- Lexicon summary -- ###

def dic_score(word):  

    file = open("lexicon/"+lexicon+"/lexicon.txt","r")

    for line in file:

        value = line.split("||")

        if word == value[2].strip() :
            
            try :
            
                return float(value[0])
            
            except :
                
                return 0 
            
def dic_word(sentence):
    
    blob = sentence
    
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0
    
    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative +=1                
                
                if result == 0 :
                    
                    neutral +=1
                
                if result > 0 :
                    
                    positive +=1
         
        sentence = (positive,negative,neutral)
                
    return sentence    

def dic_sentiment(sentence):
    
    blob = sentence
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0 
    
    total = 0

    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative = negative + float(result)                
                
                if result == 0 :
                    
                    neutral = neutral + float(result)
                
                if result > 0 :
                    
                    positive = positive + float(result)                    
                                             
        sentence = positive,negative
                
    return sentence

def dic_word_sum(sentence):
    
    blob = sentence
    text = Text(blob.lower())
    
    positive = 0
    negative =  0
    neutral = 0
    
    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0
                
                if result < 0 : 
                    
                    negative +=1
                
                if result == 0 :
                    
                    neutral +=1
                
                if result > 0 :
                    
                    positive +=1
        
        sentence = positive+negative+neutral
                
    return sentence

def dic_sentiment_sum(sentence):
    
    blob = sentence
    text = Text(blob.lower())

    positive = 0
    negative =  0
    neutral = 0 
    total = 0

    if sentence != '' :

        for w in text.words:

                result = dic_score(w) 

                if result is None :

                    result = 0 
                
                if result < 0 : 
                    
                    negative = negative + float(result) 
                
                if result == 0 :
                    
                    neutral = neutral + float(result) 
                
                if result > 0 :
                    
                    positive = positive + float(result) 
        
        sentence = positive+negative
                
    return sentence

def fraction_sum(word):
    
    try : 
    
        total = (dic_sentiment_sum(word)/dic_word_sum(word))
    
    except:
        
        total = 0
    
    return total


#### การเรียกใช้งาน API ####

Url  : http://52.77.247.131:5000/api/sentiment/v1.0.0/iuferutlkfu34fs

data type : json
parameter : text

ตัวอย่าง :  {"text" : "Hello"} 

ผลที่ได้รับ : 

{
    "Hello": [
        {
            "score": -0.3013698630136986, 
            "sentiment": "neutral"
        }
    ]
}

#### Command 
##### jupyter nbconvert --to notebook --execute senti_pos.ipynb




In [ ]:
app = Flask(__name__)
api = Api(app)
app.config['JSON_AS_ASCII'] = False

@app.route('/api/sentiment/v1.0.0/<uuid>', methods=['GET', 'POST'])
def go_sentiment(uuid):
    content = request.json
    # Token id, Call me
    token = "iuferutlkfu34fs" # uuid = token_key
#     onitext = content['text']
    onitext = content['text']
    text = Text(onitext)
    pt = ""
    total = 0 

    # for w in text.words: 
    #    	# text_s = ('%s -->%d'%(w,w.polarity))
    # total+=w.polarity
    count_str = len(text.words)

    for w in text.words:
        total+=w.polarity

    print (total)

    if total > 0:

        point = "positive"

    elif total == 0:

        point = "neutral"

    elif total < 0:

        point = "negative"
     
    if uuid == token:

        sentiment = [
        {
          # 'id': '0',
          # 'keyword':'NULL',
          # 'score1':'',
#           'score':'%.4f'%(total*1.0/count_str),
          'score':fraction_sum(onitext),
          'dic_word':dic_word(onitext),
          'sentiment':point
        }
        ]

        return jsonify({onitext:sentiment})

    else : 

        return jsonify({"id":"NULL"})


if __name__ == '__main__':
    # app.run(host= '0.0.0.0',debug=True)
    app.run(port=5000, threaded=True, host=('0.0.0.0'))